# Plotting

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rxavier/poniard/blob/master/examples/04._plotting.ipynb)

This notebook demonstrates Poniard's plotting capabilities.

If you don't have it installed, please install from PyPI.

In [1]:
# %pip install poniard

Poniard uses Plotly as a backend to plot metrics.

All plots are available at the `plot` accessor.

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from poniard import PoniardClassifier

X, y = fetch_openml("irish", return_X_y=True, as_frame=True)
y = LabelEncoder().fit_transform(y)
clf = PoniardClassifier(estimators=[LogisticRegression(),
                                    KNeighborsClassifier(),
                                    RandomForestClassifier()],
                        numeric_threshold=0.05).setup(X, y)
clf.fit()
clf.get_results()

Target info
-----------
Type: binary
Shape: (500,)
Unique values: 2

Main metric
-----------
roc_auc

Thresholds
----------
Minimum unique values to consider a feature numeric: 25
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,DVRT,,Sex,
1,Prestige_score,,Educational_level,
2,,,Type_school,


Completed: 100%|██████████| 4/4 [00:01<00:00,  3.52it/s]             


,test_roc_auc,test_accuracy,test_precision,test_recall,test_f1,fit_time,score_time
KNeighborsClassifier,0.990550,0.970,0.948261,0.986465,0.966878,0.004550,0.073748
RandomForestClassifier,0.989532,0.986,0.973816,0.995556,0.984419,0.064304,0.011957
LogisticRegression,0.987589,0.986,0.973623,0.995455,0.984369,0.009111,0.005820
DummyClassifier,0.500000,0.556,0.000000,0.000000,0.000000,0.004511,0.005028


## Plots based on the fit process.

Metrics obtained directly from a call to `fit()` can be visualized with `plot.metrics()`.

This method can plot multiple metrics, showing fold averages and error bars, or a scatter plot with the score for each fold and the mean score.

In [2]:
clf.plot.metrics(metrics=["roc_auc", "f1"])

In [3]:
clf.plot.metrics(metrics=["recall", "precision"], kind="bar")

`plot.overfitness()` shows a measure of how much each estimator is overfitting when comparing test and train scores.

In [4]:
clf.plot.overfitness(metric="f1")

## Based on cross validated predictions

Some of the plots require that predictions are obtained for the relevant estimators. These methods will automatically run the predictions using scikit-learn's `cross_val_predict()`.

Each of these will add keys to the cross validation results dict created when calling `fit()` which will be used for the plots that require them.

In [5]:
clf.plot.confusion_matrix("KNeighborsClassifier")

Completed: 100%|██████████| 1/1 [00:00<00:00, 20.27it/s]   


In [6]:
from sklearn.datasets import load_breast_cancer
from poniard import PoniardClassifier

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
clf = PoniardClassifier().setup(X, y)
clf.fit()
selected_clf = ["LogisticRegression", "XGBClassifier"]
clf.plot.roc_curve(estimator_names=selected_clf)

Target info
-----------
Type: binary
Shape: (569,)
Unique values: 2

Main metric
-----------
roc_auc

Thresholds
----------
Minimum unique values to consider a feature numeric: 56
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,mean radius,,,
1,mean texture,,,
2,mean perimeter,,,
3,mean area,,,
4,mean smoothness,,,
5,mean compactness,,,
6,mean concavity,,,
7,mean concave points,,,
8,mean symmetry,,,
9,mean fractal dimension,,,


Completed: 100%|██████████| 1/1 [00:00<00:00,  2.73it/s]


## Visualizations including computations

Some visualizations will make additional computations as part of their methods. This is because it would not be reasonable to compute them ahead of time.

`plot.permutation_importance()` takes an estimator name and outputs each feature's permutation importance.

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from poniard import PoniardClassifier

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
clf = PoniardClassifier(estimators=[KNeighborsClassifier()], metrics="f1").setup(X, y)
clf.fit()
clf.plot.permutation_importance("KNeighborsClassifier")

Target info
-----------
Type: binary
Shape: (569,)
Unique values: 2

Main metric
-----------
f1

Thresholds
----------
Minimum unique values to consider a feature numeric: 56
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,mean radius,,,
1,mean texture,,,
2,mean perimeter,,,
3,mean area,,,
4,mean smoothness,,,
5,mean compactness,,,
6,mean concavity,,,
7,mean concave points,,,
8,mean symmetry,,,
9,mean fractal dimension,,,


Completed: 100%|██████████| 2/2 [00:00<00:00, 21.93it/s]   


`plot.partial_dependence()` shows the average feature partial dependence for a given feature.

In [8]:
clf.plot.partial_dependence(estimator_name="KNeighborsClassifier",
                            feature="worst texture")